In [3]:
import pandas as pd 
from typing import Dict, Optional, List
from itertools import combinations
from collections import Counter

In [4]:
lotto = pd.read_csv("./output/merged_lotto.csv").drop_duplicates(subset=["date"])
lotto_plus = pd.read_csv("./output/merged_lottoplus.csv").drop_duplicates(subset=["date"])
mini_lotto = pd.read_csv("./output/mini_lotto.csv").drop_duplicates(subset=["date"])

In [5]:
def get_all_numbers(dataset) -> List:
    all_numbers = []
    for numbers_str in dataset['numbers']:
        numbers_list = numbers_str.split(',')
        all_numbers.extend(map(int, numbers_list))
    
    return all_numbers

In [6]:
def get_most_common_numbers(dataset, num_results: int = 6) -> Dict[Optional[int], Optional[int]]:
    all_numbers = get_all_numbers(dataset)
    
    counts_df = pd.DataFrame(all_numbers, columns=['number']).groupby('number').size().reset_index(name='count')
    
    counts_df = counts_df.sort_values(by='count', ascending=False)
    
    return counts_df.head(num_results)


In [7]:
get_most_common_numbers(lotto)

,number,count
16,17,811
37,38,798
33,34,787
23,24,782
20,21,781
3,4,778


In [8]:
get_most_common_numbers(lotto_plus)

,number,count
43,44,256
39,40,255
38,39,254
7,8,244
28,29,243
16,17,242


In [9]:
get_most_common_numbers(mini_lotto)

,number,count
35,36,816
20,21,806
27,28,792
6,7,791
3,4,790
32,33,774


In [10]:
 
def get_most_common_numbers_by_year(dataset, num_result: int = 6):
    dataset['year'] = pd.to_datetime(dataset['date'], format='%d-%m-%Y').dt.year
    
    result_df = pd.DataFrame(columns=['year', 'number', 'count'])
    
    for year, group in dataset.groupby('year'):
        all_numbers = []
        for numbers_str in group['numbers']:
            numbers_list = numbers_str.split(',')
            all_numbers.extend(map(int, numbers_list))
        
        counts_df = pd.DataFrame(all_numbers, columns=['number']).groupby('number').size().reset_index(name='count')
        counts_df = counts_df.sort_values(by='count', ascending=False)
        counts_df['year'] = year
        result_df = pd.concat([result_df, counts_df.head(num_result)])
    
    return result_df


In [11]:
get_most_common_numbers_by_year(lotto)

,year,number,count
7,1957,8,10
18,1957,19,9
12,1957,13,9
31,1957,32,8
47,1957,48,8
...,...,...,...
1,2024,2,9
25,2024,26,8
11,2024,12,8
43,2024,46,7


In [12]:
get_most_common_numbers_by_year(lotto_plus)

,year,number,count
39,2012,40,12
26,2012,27,10
42,2012,43,9
11,2012,12,9
43,2012,44,8
...,...,...,...
28,2024,29,8
7,2024,8,8
8,2024,9,8
36,2024,37,8


In [13]:
get_most_common_numbers_by_year(mini_lotto)

,year,number,count
25,1981,26,11
4,1981,5,11
33,1981,34,10
29,1981,30,9
14,1981,15,8
...,...,...,...
40,2024,41,18
17,2024,18,18
27,2024,28,16
30,2024,31,15


In [14]:
def most_common_number_string(dataset, min_length: int = 2, max_length: int = 6):
    if min_length < 2:
        raise ValueError("min_length value must be greater than 2")
    
    set_counts = Counter()
   
    numbers_split = dataset['numbers'].str.split(',')
    
    for nums in numbers_split:
        nums.sort()
        for length in range(min_length, min(max_length, len(nums)) + 1):
            sets = [tuple(sorted(combo)) for combo in combinations(nums, length)]
            set_counts.update(sets)
    
    sets_df = pd.DataFrame(list(set_counts.items()), columns=['Set', 'Count'])
    sets_df = sets_df.sort_values(by='Count', ascending=False)
    
    return sets_df

In [19]:
most_common_number_string(lotto)

,Set,Count
406,"(21, 39)",105
1007,"(1, 17)",102
8647,"(46, 7)",101
3179,"(29, 9)",101
5388,"(17, 26)",100
...,...,...
61659,"(27, 3, 43, 44)",1
61658,"(25, 43, 44, 47)",1
61657,"(25, 3, 44, 47)",1
61656,"(25, 3, 43, 47)",1


In [16]:
most_common_number_string(lotto_plus)

,Set,Count
292,"(29, 6)",39
2388,"(39, 49)",38
1562,"(17, 29)",38
4753,"(38, 40)",38
790,"(12, 8)",37
...,...,...
24019,"(2, 21, 26, 38)",1
24020,"(2, 21, 26, 47)",1
24021,"(2, 21, 37, 38)",1
24022,"(2, 21, 37, 47)",1


In [17]:
most_common_number_string(mini_lotto)

,Set,Count
921,"(32, 7)",104
84,"(36, 42)",102
2648,"(2, 3)",101
2235,"(21, 36)",100
896,"(28, 30)",95
...,...,...
25463,"(26, 35, 38, 5)",1
25462,"(14, 35, 38, 5)",1
25461,"(14, 26, 38, 5)",1
25460,"(14, 26, 35, 5)",1


In [18]:
def test_most_common_number_string() -> None:
    test_number_string = [15, 24, 27, 37, 46, 8]
    test_number_set = set(test_number_string)

    test_number_set = set(test_number_string)

    def check_existence(row):
        row_numbers = set(map(int, row.split(',')))
        return test_number_set == row_numbers

    test_existence = lotto['numbers'].apply(check_existence)
    
    return test_existence.sum() == 1
    
test_most_common_number_string()
    

True